In [1]:

# Solution:
import re
import jinja2
from collections import namedtuple
from itertools import chain

"""
The format is:
(ccy)(swap_type)[forward term](maturity)
Ccy can be a list of configurable currencies, eg 'us', 'eu', 'cd'...
Similarly, swap_type can be 'sw', 'ois', 'ff' etc.
Maturity can be '3m', '6m', '1y', '10y', '30y' etc
Forward term is optional, and have same format as Maturity.
"""

class TemplateBase(object):
    @classmethod
    def _flattemplate(cls):
        return '|'.join(cls.templates) if isinstance(cls.templates, (list, tuple)) else cls.templates

    @classmethod
    def jtemplate(cls):
        return jinja2.Template(cls._flattemplate())

    @classmethod
    def _vars(cls):
        if isinstance(cls.templates, (list, tuple)):
            vars = [re.findall("\{\{(\w+)\}\}+", temp) for temp in cls.templates]
            return list(chain(*vars))
        else:
            return re.findall("\{\{(\w+)\}\}+", cls.templates)

    @classmethod
    def _pattern(cls):
        vars = cls._vars()
        fills = ['(%s)' % "|".join(getattr(cls, v)) for v in vars]
        pattern = "^" + cls.jtemplate().render(dict(zip(vars, fills))) + "$"
        #print(f'{cls.__name__} The pattern is:', pattern)
        return re.compile(pattern, re.I)

    @classmethod
    def matches(cls, sym):
        res = cls._pattern().match(sym)
        if res:
            vars = cls._vars()
            matchedresults = namedtuple('matchedresults', vars)
            for i, v in enumerate(res.groups()):
                setattr(matchedresults, vars[i], v)
            f = getattr(cls, 'validate', None)
            if f and not f(matchedresults):
                return False
            return matchedresults
        return False


class SwapTemplate(TemplateBase):
    templates = "{{ccy}}{{swap_type}}{{forward}}{{maturity}}"
    ccy = ['us', 'ussw', 'eu', 'cd', 'bp', 'jy', 'mx', 'cny', 'ewy', 'au']
    swap_type = ['sw', '15y', 'ois', 'ff', 'fedfund']
    forward = ['[0-9]+[BWMY]', '']
    maturity = ['[0-9]+[BWMY]']


class FXTemplate(TemplateBase):
  templates = "{{ccy1}}{{ccy2}}"
  ccy1 = ['usd', 'cad', 'jpy', 'eur', 'gbp']
  ccy2 = ['usd', 'cad', 'jpy', 'eur', 'gbp']
  
  @staticmethod
  def validate(res):
    return res.ccy1.upper() != res.ccy2.upper()


class MtgeTemplate(TemplateBase):
  templates = 'mtge_{{maturity}}'
  maturity = ['[0-9]+[BWMY]']
  @staticmethod
  def validate(res):
    return int(res.maturity[:-1]) <= 30

class EquityTemplate(TemplateBase):
  pass


class Ticker(object):
    def __init__(self, typ, res):
        self.typ = typ
        self.fields = res._fields
        for field in self.fields:
            setattr(self, field, getattr(res, field))


REGISTRY = [('IRSwap', SwapTemplate),
            ('FX', FXTemplate), 
            ('Mtge', MtgeTemplate)
            ]

class TickerMatcher():
  @staticmethod
  def identify_ticker(ticker):
      for typ, temp in REGISTRY:
          res = temp.matches(ticker)
          if res:
              return Ticker(typ, res)
  
  def match(self, ticker):
      res = self.identify_ticker(ticker)
      if res:
          strList = ['### %s matched successfully with %s template ###' % (ticker, res.typ)]
          for field in res.fields:
              strList.append('%s:\t%s' % (field, getattr(res, field)))
      else:
          strList = ['*** %s can NOT match with any template! ***' % ticker]

      return '\n'.join(strList)


# Ticker Input
matcher = TickerMatcher()
for ticker in ('ussw15y10y', 'mxxxx10y', 'usois10y', 'usois10yy20y', 'bpff5y10y', 'ussw10x', 'USDEUR', 'CADJPY', 'usdusd', 'usdUSD', 'mtge_10y', 'mtge_90y'):
    s = matcher.match(ticker)
    print(s+'\n')



### ussw15y10y matched successfully with IRSwap template ###
ccy:	us
swap_type:	sw
forward:	15y
maturity:	10y

*** mxxxx10y can NOT match with any template! ***

### usois10y matched successfully with IRSwap template ###
ccy:	us
swap_type:	ois
forward:	
maturity:	10y

*** usois10yy20y can NOT match with any template! ***

### bpff5y10y matched successfully with IRSwap template ###
ccy:	bp
swap_type:	ff
forward:	5y
maturity:	10y

*** ussw10x can NOT match with any template! ***

### USDEUR matched successfully with FX template ###
ccy1:	USD
ccy2:	EUR

### CADJPY matched successfully with FX template ###
ccy1:	CAD
ccy2:	JPY

*** usdusd can NOT match with any template! ***

*** usdUSD can NOT match with any template! ***

### mtge_10y matched successfully with Mtge template ###
maturity:	10y

*** mtge_90y can NOT match with any template! ***



In [2]:

# inspired by "Quantlib Python Cookbook" by Balaraman and Ballabio
from QuantLib import *
def calculate_swap():
  today = Date(31, December, 2018)
  Settings.instance().evaluationDate = today

  quotes = [SimpleQuote(2.80763/100) ]

  helpers = [DepositRateHelper(QuoteHandle(quotes[0]), Period(3,Months), 3, UnitedStates(), ModifiedFollowing, False, Actual360())]

  for rate, tenor in [(2.6569, 2), (2.5899, 3), (2.5682, 4), (2.5703, 5),
                      (2.5969, 6), (2.6235, 7), (2.6493, 8), (2.6828, 9),
                      (2.7084, 10), (2.7309, 11)]:

      quotes.append(SimpleQuote(rate/100))
      helpers.append(SwapRateHelper(QuoteHandle(quotes[-1]),
                    Period(tenor, Years), UnitedStates(),
                    Annual, Unadjusted, Thirty360(Thirty360.BondBasis),USDLibor(Period(3,Months))))

  rate_curve = PiecewiseLogCubicDiscount(2, UnitedStates(), helpers, Actual365Fixed())
  curve_handle = RelinkableYieldTermStructureHandle(rate_curve)


  fixed_schedule = Schedule(Date(1, February, 2019), Date(1, February, 2029),
  Period(1, Years), UnitedStates(), ModifiedFollowing, ModifiedFollowing,
  DateGeneration.Forward, False)

  floating_schedule = Schedule(Date(1, February, 2019), Date(1, February, 2029),
  Period(6, Months), UnitedStates(), ModifiedFollowing, ModifiedFollowing,
  DateGeneration.Forward, False)

  index = USDLibor(Period(3,Months),curve_handle)
  swap = VanillaSwap(VanillaSwap.Payer, 10000000.0,fixed_schedule, 2.5/100, Thirty360(), floating_schedule, index, 0.0, Actual360())

  swap.setPricingEngine(DiscountingSwapEngine(curve_handle))
  P0 = swap.NPV()
  return P0, swap


P0, swap = calculate_swap()
print("Swap NPV=", P0)




Swap NPV= 180735.73676135624
